In [1]:
import os
from dotenv import load_dotenv


from typing import Annotated
from openai import AsyncOpenAI


from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory


from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

from semantic_kernel.connectors.ai import FunctionChoiceBehavior

from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

In [4]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), base_url="https://models.inference.ai.azure.com/")


chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
  
)
kernel.add_service(chat_completion_service)

In [6]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""


agent = ChatCompletionAgent(
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
)

In [7]:
from IPython.display import display, HTML


async def main():
    # Define the chat history
    chat_history = ChatHistory()

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 2 kids from Singapore to Melboune",
    ]

    for user_input in user_inputs:
        # Add the user input to the chat history
        chat_history.add_user_message(user_input)

        # Start building HTML output
        html_output = f"<div style='margin-bottom:10px'>"
        html_output += f"<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += f"</div>"

        agent_name: str | None = None
        full_response = ""

        # Collect the agent's response
        async for content in agent.invoke_stream(chat_history):
            if not agent_name and hasattr(content, 'name'):
                agent_name = content.name

            # Add content to response
            if hasattr(content, 'content') and content.content and content.content.strip():
                full_response += content.content

        # Add agent response to HTML
        html_output += f"<div style='margin-bottom:20px'>"
        html_output += f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
        html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{full_response}</div>"
        html_output += f"</div>"
        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()